In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)


1.15.0
sys.version_info(major=3, minor=7, micro=6, releaselevel='final', serial=0)
matplotlib 3.1.3
numpy 1.18.1
pandas 1.0.0
sklearn 0.22.1
tensorflow 1.15.0
tensorflow.python.keras.api._v1.keras 2.2.4-tf


## 读取数据并进行预处理

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(x_train_all, y_train_all), (x_test, y_test) = fashion_mnist.load_data()
x_valid, x_train = x_train_all[:5000], x_train_all[5000:]
y_valid, y_train = y_train_all[:5000], y_train_all[5000:]

print(x_valid.shape, y_valid.shape)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(5000, 28, 28) (5000,)
(55000, 28, 28) (55000,)
(10000, 28, 28) (10000,)


In [3]:
print(np.max(x_train), np.min(x_train))

255 0


In [4]:
# x = (x - u) / std

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# x_train: [None, 28, 28] -> [None, 784]
x_train_scaled = scaler.fit_transform(
    x_train.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)
x_valid_scaled = scaler.transform(
    x_valid.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)
x_test_scaled = scaler.transform(
    x_test.astype(np.float32).reshape(-1, 1)).reshape(-1, 28 * 28)

y_train = np.asarray(y_train, dtype = np.int64)   # 更改数据类型
y_valid = np.asarray(y_valid, dtype = np.int64)
y_test = np.asarray(y_test, dtype = np.int64)

In [5]:
print(np.max(x_train_scaled), np.min(x_train_scaled))

2.0231433 -0.8105136


### 创建dataset   make_one_shot_iterator

In [12]:
def make_dataset(images, labels, epochs, batch_size, shuffle = True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [7]:
# # 此处遍历只有在eager excution下才会被被支持
# batch_size = 20
# epochs = 10
# dataset = make_dataset(x_train_scaled, y_train, epochs=epochs, batch_size=batch_size)
# for data, label in dataset.take(1):
#     print(data)
#     print(label)

In [13]:
batch_size = 20
epochs = 10
dataset = make_dataset(x_train_scaled, y_train, epochs=epochs, batch_size=batch_size)
dataset_iter = dataset.make_one_shot_iterator()  # 创建一个遍历器, 特点：自动初始化、不能被重新初始化
x, y = dataset_iter.get_next()   # x, y是tensor，只有在打开session run时才能获取具体的值
# with tf.Session() as session:
#     x_val, y_val = session.run([x, y])
#     print(x_val.shape)
#     print(y_val.shape)


### 创建dataset make_initializable_iterator ： 可以通过session run initializer 更改数据源

In [ ]:
def make_dataset(images, labels, epochs, batch_size, shuffle = True):
    dataset = tf.data.Dataset.from_tensor_slices((images, labels))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset

In [ ]:
batch_size = 20
epochs = 10

images_placeholder = tf.placeholder(tf.float32, [None, 28 * 28])
labels_placeholder = tf.placeholder(tf.int64, [None, ])

dataset = make_dataset(images_placeholder, labels_placeholder, epochs=epochs, batch_size=batch_size)
dataset_iter = dataset.make_initializable_iterator()  # 创建一个遍历器, 特点：自动初始化、不能被重新初始化
x, y = dataset_iter.get_next()   # x, y是tensor，只有在打开session run时才能获取具体的值
with tf.Session() as session:
    session.run(dataset_iter.initializer, 
               feed_dict = {
                   images_placeholder: x_train_scaled,
                   labels_placeholder: y_train
               })
    x_val, y_val = session.run([x, y])
    print(x_val.shape)
    print(y_val.shape)
    
    session.run(dataset_iter.initializer, 
               feed_dict = {
                   images_placeholder: x_valid_scaled,
                   labels_placeholder: y_valid
               })
    x_val, y_val = session.run([x, y])
    print(x_val.shape)
    print(y_val.shape)


## 使用tensorflow1进行模型搭建（以下代码是针对make_one_shot_iterator迭代器的）

### 构建图

In [9]:
# 网络结构
hidden_units = [100, 100]
class_num = 10
# x = tf.placeholder(tf.float32, [None, 28 * 28])
# y = tf.placeholder(tf.int64, [None])

input_for_next_layer = x
for hidden_unit in hidden_units:
    input_for_next_layer = tf.layers.dense(input_for_next_layer, hidden_unit, activation=tf.nn.relu)
logits = tf.layers.dense(input_for_next_layer, class_num)
# sparse_softmax_cross_entropy函数作用
# 1. logit -> softmax -> prob
# 2. labels -> one_hot
# 3. calculate cross entropy
loss = tf.losses.sparse_softmax_cross_entropy(labels=y, logits=logits)

# 获取accuracy
prediction = tf.argmax(logits, 1)
correct_prediction = tf.equal(prediction, y)
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float64))

# train_op 用于训练模型 每调用一次相当于train一次
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

Instructions for updating:
Use keras.layers.Dense instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [10]:
print(x)   # 此时x是iter的tensor而不是placeholder的tensor
print(logits)

Tensor("IteratorGetNext:0", shape=(?, 784), dtype=float32)
Tensor("dense_2/BiasAdd:0", shape=(?, 10), dtype=float32)


### 运行图

In [11]:
# 使用session与图进行交互
# 先定义一些变量
init = tf.global_variables_initializer()
# batch_size = 20
# epochs = 10
train_steps_per_epoch = x_train.shape[0] // batch_size
# valid_steps = x_valid.shape[0] // batch_size

# # 验证
# def eval_with_session(session, x, y, accuracy, images, labels, batch_size):
#     eval_steps = images.shape[0] // batch_size
#     eval_accuracies = []
#     for step in range(eval_steps):
#         batch_data = images[step * batch_size : (step + 1)* batch_size]
#         batch_label = labels[step * batch_size : (step + 1)* batch_size]
#         accuracy_val = session.run(accuracy, feed_dict = {
#             x: batch_data,
#             y: batch_label
#         })
#         eval_accuracies.append(accuracy_val)
#         return np.mean(eval_accuracies)

# 打开session
with tf.Session() as session:
    session.run(init)   # 运行初始化，之后图才能被构建和调用
    for epoch in range(epochs):
        for step in range(train_steps_per_epoch):
#             batch_data = x_train_scaled[step * batch_size : (step + 1)* batch_size]
#             batch_label = y_train[step * batch_size : (step + 1)* batch_size]
            # loss, accuracy, train_op 都是算子，且可以一起运算
            loss_val, accuracy_val, _ = session.run([loss, accuracy, train_op]
#                                                     feed_dict = {  # feed_dict将placeholder与数据对应起来
#                                                                 x: batch_data,
#                                                                 y: batch_label }
                                                    )
            print('\r[Train] epoch: %d, step: %d, loss: %3.3f, accuracy: %2.2f' % (epoch, step, loss_val, accuracy_val), end=" ")
#         valid_accuracy = eval_with_session(session, x, y, accuracy, x_valid_scaled, y_valid, batch_size)
#         print('\t[Valid] acc: %2.2f' % (valid_accuracy))

[Train] epoch: 9, step: 2749, loss: 0.195, accuracy: 0.90  